# **Data Mining I | <span style="color: darkseagreen;">Exploratory Data Analysis and Pre-Processing</span>**

## A Clustering-Based Exploration of WorkplaceAbsence Patterns

*Group 07*

*Group members* : Francisco Gomes (20221810), Margarida Marchão, Pedro Coimbras and Marta Alves.

<a id="1"></a>

# **1. Importing the datasets and libraries <font>**

In [1]:
#imports
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from pylab import rcParams

from utils1 import (
    calculate_percentage,
    analyze_missingness_patterns,
    diagnose_seasons_from_month
)
pd.set_option("display.max_columns", None)

In [2]:
#from google.colab import drive
# drive.mount('/content/drive', force_remount=True)
# path = '/content/drive/MyDrive/Colab Notebooks/DM1/projeto/data/'
path2 = '/Users/franciscogomes/Desktop/Faculdade/Master/Data Mining/Project/reasons_for_absenteeism/data/'

<a id="1"></a>

# **2. Preliminary Data Analysis<font>**

In [3]:
pd.set_option("display.max_columns", None)
abs = pd.read_csv(path2 + 'absenteeism_data.csv',delimiter=';')
abs = abs.set_index("ID")
abs.head()


,Reason for absence,Month of absence,Day of the week,Seasons,Days since previous absence,Transportation expense,Distance from Residence to Work,Estimated commute time,Service time,Years until retirement,Date of Birth,Disciplinary failure,Education,Number of children,Social drinker,Social smoker,Number of pets,Weight,Height,Body mass index,Absenteeism time in hours
ID,,,,,,,,,,,,,,,,,,,,,
11,Unjustified absence,July,Tuesday,Summer,0.0,289,36,69,13,32,1992-08-15,No,1,2,Y,No,1,90,172,30,4
36,Unspecified,July,Tuesday,NaN,0.0,118,13,26,18,15,1975-09-02,Yes,1,1,Y,No,0,98,178,31,0
3,Medical consultation,July,Wednesday,Summer,0.0,179,51,108,18,27,1987-04-08,No,1,0,Yes,No,0,89,170,31,2
7,Diseases of the eye and adnexa,July,Thursday,NaN,0.0,279,5,5,14,26,1986-07-25,No,1,2,Yes,Yes,0,68,168,24,4
11,Medical consultation,July,Thursday,Summer,0.0,289,36,69,13,32,1992-08-15,No,1,2,Yes,No,1,90,172,30,2


#### **Feature Groups Overview**

| **Category** | **Feature Name** | **Description / Notes** |
|---------------|------------------|--------------------------|
| **Absence Information** | Reason for absence | Registered reason for each absence |
|  | Month of absence | Month when the absence occurred |
|  | Day of the week | Weekday of the absence (Monday–Friday) |
|  | Seasons | Season in which the absence occurred |
|  | Days since previous absence | Number of days since last absence |
|  | Absenteeism time in hours | Duration of absence (in hours) |
| **Commute & Transportation** | Transportation expense | Monthly commuting expenses (BRL) |
|  | Distance from Residence to Work | Distance between home and workplace |
|  | Estimated commute time | Estimated duration of daily commute |
| **Work-Related Features** | Service time | Years worked at the company |
|  | Years until retirement | Years remaining until retirement eligibility |
|  | Disciplinary failure | Whether absence violated company policy (binary) |
| **Demographics & Personal Info** | Date of Birth → *Age* | Employee’s date of birth (convert to age) |
|  | Education | Education level (1–High School → 4–Master’s/PhD) |
|  | Number of children | Total number of children |
|  | Social smoker | Smokes socially (binary) |
|  | Social drinker | Drinks socially (binary) |
|  | Number of pets | Number of pets owned |
| **Health & Physical Attributes** | Weight | Weight (kg) |
|  | Height | Height (cm) |
|  | Body mass index (BMI) | Calculated from height and weight |


okay, lets gOOooOOoOOo

In [4]:
display(abs.shape)

(800, 21)

| **Type** | **Count** |
|-----------|-----------|
| Observations (rows) | 800 |
| Variables (columns) | 21 |

Each record corresponds to **one employee’s absence instance**, including personal, work, commute, and health-related information.



<a id="1"></a>

### **2.1. Data Types <font>**

In [5]:
display(abs.info())

<class 'pandas.core.frame.DataFrame'>
Index: 800 entries, 11 to 35
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Reason for absence               740 non-null    object 
 1   Month of absence                 737 non-null    object 
 2   Day of the week                  740 non-null    object 
 3   Seasons                          573 non-null    object 
 4   Days since previous absence      781 non-null    float64
 5   Transportation expense           800 non-null    int64  
 6   Distance from Residence to Work  800 non-null    int64  
 7   Estimated commute time           800 non-null    int64  
 8   Service time                     800 non-null    object 
 9   Years until retirement           800 non-null    int64  
 10  Date of Birth                    800 non-null    object 
 11  Disciplinary failure             800 non-null    object 
 12  Education                  

None

 - missing values in: reason of absence, month of absence, day of the week, seasons, days since previous absence
 - education is int
 - we can predict seasons with month of absence
 - transportation expense is in brasilian reais -> mudar para euros
 - service time and weight has missing values '-' -> mudar para int
 - one hot no disciplinary failure, social frinker and smoker
 - social drinker and smoker has values of 'Y'
 - ver 0s na coluna do absenteeism time in hours -> 0h can mean 30min or it wasnt absent
 - maybe mudar int64 para int32
 - dar drop do days since previous absence porque só tem um unico valor que é 0.0
 - ver outliers
 - compute age
 - há duplicates - 48

<a id="1"></a>

### **2.4. Missing Values Analysis <font>**

In [6]:
abs.isna().sum()

Reason for absence                  60
Month of absence                    63
Day of the week                     60
Seasons                            227
Days since previous absence         19
Transportation expense               0
Distance from Residence to Work      0
Estimated commute time               0
Service time                         0
Years until retirement               0
Date of Birth                        0
Disciplinary failure                 0
Education                            0
Number of children                   0
Social drinker                       0
Social smoker                        0
Number of pets                       0
Weight                               0
Height                               0
Body mass index                      0
Absenteeism time in hours            0
dtype: int64

In [7]:
missing_columns = ['Reason for absence', 'Month of absence', 'Day of the week', 'Seasons', 'Days since previous absence']
missing_percentages = calculate_percentage(abs, columns=missing_columns)

for col, percent in missing_percentages.items():
    display(f'Around {percent}% of the data points in the column {col} are missing values')

'Around 7.5% of the data points in the column Reason for absence are missing values'

'Around 7.9% of the data points in the column Month of absence are missing values'

'Around 7.5% of the data points in the column Day of the week are missing values'

'Around 28.4% of the data points in the column Seasons are missing values'

'Around 2.4% of the data points in the column Days since previous absence are missing values'

estamos a ver o porque de algumas variaves serem object ------------------------------------------------------------------------------

| **Column** | **Missing Values** | **Missing (%)** | **Notes / Observations** |
|-------------|--------------------|-----------------|---------------------------|
| Reason for absence | 60 | 7.5% | Moderate number of missing entries — may relate to data entry issues or unclassified absences |
| Month of absence | 63 | 7.9% | Slightly higher missing ratio; may correspond to missing absence dates |
| Day of the week | 60 | 7.5% | Matches missing patterns in "Reason for absence" — possibly same missing records |
| Seasons | 227 | 28.4% | **Highest missing rate**; may require imputation or removal |
| Days since previous absence | 19 | 2.4% | Minor; mostly zeros, might not significantly impact clustering |
| **All other columns** | 0 | 0% | Fully complete |

In [8]:
cols_absence = ['Reason for absence', 'Month of absence', 'Day of the week', 'Seasons', 'Days since previous absence']
missing_summary = analyze_missingness_patterns(abs, cols_absence)

Pairwise Missingness Correlation Matrix:


,Reason for absence,Month of absence,Day of the week,Seasons,Days since previous absence
Reason for absence,1.00,0.97,1.00,0.45,-0.04
Month of absence,0.97,1.00,0.97,0.45,-0.05
Day of the week,1.00,0.97,1.00,0.45,-0.04
Seasons,0.45,0.45,0.45,1.00,-0.03
Days since previous absence,-0.04,-0.05,-0.04,-0.03,1.00



 Rows with all of Reason/Month/Day missing: 60

 Top 10 Pairs with Highest Overlapping Missing Values:


count
Month of absence   Seasons                         62
Reason for absence Month of absence                60
                   Day of the week                 60
                   Seasons                         60
Month of absence   Day of the week                 60
Day of the week    Seasons                         60
Seasons            Days since previous absence      4
Reason for absence Days since previous absence      0
Month of absence   Days since previous absence      0
Day of the week    Days since previous absence      0


Example indices where Reason/Month/Day are all missing:


Index([20, 20, 3, 10, 14, 3, 20, 15, 34, 28, 34, 20, 33, 17, 28, 36, 3, 20, 11,
       11],
      dtype='int64', name='ID')

### Missingness Correlation Analysis

| Variable Pair | Correlation | Interpretation |
|----------------|-------------|----------------|
| Reason ↔ Month | 0.97 | Strongly correlated missingness |
| Reason ↔ Day | 1.00 | Identical missing rows |
| Month ↔ Day | 0.97 | Same missing pattern |
| Reason/Month/Day ↔ Seasons | 0.45 | Partial overlap |
| Days since previous absence ↔ others | ≈ 0.0 | No relationship |

The columns `Reason for absence`, `Month of absence`, and `Day of the week` have **perfectly overlapping missing rows (n = 60)**.  
These likely represent **unlogged or incomplete absence records**.  
`Seasons` has additional missing cases, possibly due to inconsistent month data, and `Days since previous absence` appears independent and uninformative.


In [20]:
result_seasons = diagnose_seasons_from_month(abs)

Original missing 'Seasons': 227
Potentially recoverable from 'Month of absence': 165 (72.7%)
Rows where existing 'Seasons' disagrees with derived value: 366


### Diagnostic Results — Seasons Derived from Month of Absence

| **Metric** | **Value** | **Interpretation** |
|-------------|-----------|--------------------|
| Original missing `Seasons` | **227** | About 28.4% of all entries lacked a season label. |
| Potentially recoverable from `Month of absence` | **165 (≈72.7%)** | A large portion of missing `Seasons` can be reliably inferred from the corresponding month using the Brazilian seasonal mapping. |
| Rows where existing `Seasons` disagree with derived value | **366** | There are inconsistencies between the recorded `Seasons` and the season logically expected from the month. |

- Roughly **three-quarters of missing `Seasons` values** could be **reconstructed** based on the month, confirming that the variables are strongly related.  
- However, the presence of **366 mismatched cases** suggests **data quality issues**, likely due to inconsistent labeling or incorrect season assignments.  
- These discrepancies highlight the need for **careful validation** before applying imputation or transformation in the preprocessing phase.  
- At this EDA stage, we only **identify** these inconsistencies — the actual correction will be handled later during **feature engineering**.


<a id="1"></a>

### **2.5. Summary Statistics - (numerical data) <font>**

In [10]:
abs.describe().T

,count,mean,std,min,25%,50%,75%,max
Days since previous absence,781.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0
Transportation expense,800.0,221.92750,66.778732,118.0,179.0,225.0,260.0,388.0
Distance from Residence to Work,800.0,29.79875,14.875057,5.0,16.0,26.0,50.0,52.0
Estimated commute time,800.0,59.34875,31.301067,5.0,31.0,52.0,94.0,114.0
Years until retirement,800.0,26.74250,9.269498,-1.0,25.0,28.0,33.0,38.0
Education,800.0,1.28750,0.667383,1.0,1.0,1.0,1.0,4.0
Number of children,800.0,1.04375,1.120672,0.0,0.0,1.0,2.0,4.0
Number of pets,800.0,0.74500,1.323694,0.0,0.0,0.0,1.0,8.0
Height,800.0,172.09375,6.058122,163.0,169.0,170.0,172.0,196.0
Body mass index,800.0,26.64125,4.267354,19.0,24.0,25.0,31.0,38.0


### Summary of Key Numerical Insights

- **Days since previous absence:** Constant (all 0) → no variance, will likely be removed.  
- **Transportation expense / Commute distance / Commute time:** Show wide variation — important indicators of employee mobility and potential absenteeism factors.  
- **Years until retirement:** Mostly consistent but includes a **negative value** → data quality issue to fix later.  
- **Education:** Low variability (most have level 1) → limited clustering value.  
- **Children & Pets:** Mild variation; may reflect personal/lifestyle differences.  
- **BMI:** Moderate variability (mean ≈ 26.6) — possible link to health-related absences.  
- **Absenteeism time in hours:** Skewed with a few extreme values — key variable to analyze and scale properly.

#### Overall Notes
- Some variables (e.g., **Days since previous absence**, **Height**) provide little information.  
- Commute-related and absenteeism measures are **most informative** for later clustering.  
- **Scaling and cleaning** (especially for `Years until retirement`) will be required before modeling.


#### **Removing Column <font>**

In [22]:
display(abs['Days since previous absence'].unique())

array([ 0., nan])

`Days since previous absence`

From the summary statistics, we observe that:

| Metric | Value |
|---------|--------|
| Count | 781 |
| Mean | 0.00 |
| Standard Deviation | 0.00 |
| Min / Max | 0.0 / 0.0 |
| Quartiles (25%, 50%, 75%) | All 0.0 |

- The variable `Days since previous absence` shows **no variation** across observations — all values are zero.  
- This indicates that, in practice, employees are either **not having consecutive absences** or the feature is **not properly recorded** in the dataset.  
- With such low variance, this variable provides **no discriminative power** for clustering or pattern discovery.


In [23]:
abs = abs.drop(columns=['Days since previous absence'])

<a id="1"></a>

### **2.6. Findings in the Categorical Data <font>**

In [25]:
abs.describe(include = ['O']).T

,count,unique,top,freq
Reason for absence,740,28,Medical consultation,149
Month of absence,737,12,March,87
Day of the week,740,6,Monday,159
Seasons,573,4,Spring,155
Service time,800,19,18,150
Date of Birth,800,36,1987-04-08,122
Disciplinary failure,800,2,No,756
Social drinker,800,3,Yes,367
Social smoker,800,3,No,744
Weight,800,27,89,110


In [26]:
display(abs['Weight'].unique())

array(['90', '98', '89', '68', '80', '65', '-', '88', '67', '69', '86',
       '84', '58', '83', '106', '95', '73', '70', '56', '63', '76', '75',
       '77', '79', '100', '108', '94'], dtype=object)

In [12]:
display(abs['Date of Birth'].unique())

array(['1992-08-15', '1975-09-02', '1987-04-08', '1986-07-25',
       '1997-07-08', '1989-02-20', '1991-09-10', '1988-06-01',
       '1984-10-01', '1992-05-24', '1978-02-16', '1997-01-19',
       '1996-05-22', '1977-04-28', '1993-04-25', '1998-12-19',
       '1988-05-10', '1982-06-19', '1985-05-20', '1997-01-01',
       '1997-08-03', '1994-06-26', '1995-08-01', '1985-06-16',
       '1975-06-07', '1989-02-10', '1976-09-21', '1967-11-02',
       '1982-08-04', '1992-04-20', '1986-09-13', '1993-11-21',
       '1994-09-18', '1979-01-20', '1985-10-20', '1972-11-27'],
      dtype=object)

In [13]:
print(abs['Social drinker'].unique())
print(abs['Social smoker'].unique())

['Y' 'Yes' 'No']
['No' 'Yes' 'Y']


In [14]:
abs['Weight'].unique()

array(['90', '98', '89', '68', '80', '65', '-', '88', '67', '69', '86',
       '84', '58', '83', '106', '95', '73', '70', '56', '63', '76', '75',
       '77', '79', '100', '108', '94'], dtype=object)

In [15]:
#ver a questao dos 0s (8h means all day probably)
abs['Absenteeism time in hours'].value_counts(dropna=False)


Absenteeism time in hours
8      216
2      170
3      124
1       97
4       65
0       48
16      21
24      18
5        7
32       7
40       7
80       5
64       3
120      3
112      3
7        2
56       2
104      1
48       1
Name: count, dtype: int64

In [16]:
abs['Days since previous absence'].unique()

array([ 0., nan])

In [18]:
#statistics of categorical vars
abs.describe(include = ['O']).T

,count,unique,top,freq
Reason for absence,740,28,Medical consultation,149
Month of absence,737,12,March,87
Day of the week,740,6,Monday,159
Seasons,573,4,Spring,155
Service time,800,19,18,150
Date of Birth,800,36,1987-04-08,122
Disciplinary failure,800,2,No,756
Social drinker,800,3,Yes,367
Social smoker,800,3,No,744
Weight,800,27,89,110


most frequent reason for absence is medical consultation

duplicates ----------------------------------

In [19]:
abs[abs.duplicated(keep=False)]

,Reason for absence,Month of absence,Day of the week,Seasons,Days since previous absence,Transportation expense,Distance from Residence to Work,Estimated commute time,Service time,Years until retirement,Date of Birth,Disciplinary failure,Education,Number of children,Social drinker,Social smoker,Number of pets,Weight,Height,Body mass index,Absenteeism time in hours
ID,,,,,,,,,,,,,,,,,,,,,
20,NaN,NaN,NaN,NaN,0.0,260,50,94,11,29,1989-02-20,No,1,4,Yes,No,0,65,168,23,8
20,NaN,NaN,NaN,NaN,0.0,260,50,94,11,29,1989-02-20,No,1,4,Yes,No,0,65,168,23,8
3,NaN,NaN,NaN,NaN,0.0,179,51,108,18,27,1987-04-08,No,1,0,Y,No,0,89,170,31,3
34,Medical consultation,October,Tuesday,Spring,0.0,118,10,22,10,28,1988-05-10,No,1,0,No,No,0,83,172,28,3
34,Medical consultation,October,Tuesday,Spring,0.0,118,10,22,10,28,1988-05-10,No,1,0,No,No,0,83,172,28,3
10,Patient follow-up,December,Wednesday,NaN,0.0,361,52,114,3,37,1997-07-08,No,1,1,Yes,No,4,80,172,27,8
28,Medical consultation,December,Tuesday,Spring,0.0,225,26,50,9,37,1997-08-03,No,1,1,No,No,2,69,169,24,2
10,Patient follow-up,December,Wednesday,NaN,0.0,361,52,114,3,37,1997-07-08,No,1,1,Yes,No,4,80,172,27,8
28,NaN,NaN,NaN,NaN,0.0,225,26,50,9,37,1997-08-03,No,1,1,No,No,2,69,169,24,1


damn